# DRUIDs Magic on Galaxies

Some of DRUIDs strengths are demonstrated with its ability to handle nested image features. Like the details resovlable in face on galaxies.

First lets download a nice galaxy.

In [ ]:
from astrocut import fits_cut
from astropy.coordinates import SkyCoord
from astropy import units as u

# code to download a nice galaxy image
# might take a while to download as the image is 1.5 GB in size. But this will be cashed in the memory
PATH_to_data = ["https://dataportal.eso.org/dataportal_new/file/ADP.2019-02-11T13:02:26.713"]

center_coord = SkyCoord('01:21:20.458 -34:03:48.12', unit=(u.hourangle, u.deg))
tout_size = [1000,1000]  # pixels

cutout_file = fits_cut(PATH_to_data, center_coord, tout_size, memory_only=True)
cutout_file[0].info()
image = cutout_file[0][1].data

In [ ]:
from DRUID import sf

findmysources = sf(image=image,                             # Image Array
                   header=cutout_file[0][1].header,         # Image Header
                   mode='optical',                          # Mode (optical, radio, other)
                   cutup=False,                             # Cutup the image into smaller pieces?
                   smooth_sigma=1.3,                        # Sigma for the smoothing the image with gaussian filter
                   area_limit=3)                            # Minimum area for a source to be considered as a source
findmysources.set_background(detection_threshold=2,         # Detection threshold for the background
                             analysis_threshold=2,          # Analysis threshold for the background
                             mode='SEX')                    # Mode (SEX, rms, mad_std, sigma_clip) 
findmysources.phsf(lifetime_limit=findmysources.local_bg,   # Lifetime limit for the sources (in absolute units)
                   lifetime_limit_fraction=1.001)           # Lifetime limit for the sources (as fraction of brith - death)
findmysources.source_characterising()                       # Characterise the sources (i.e. measure the properties of the sources and calculate the contours)
catalogue = findmysources.catalogue                         # get the catalogue from the sf object

from matplotlib.pylab import plt
from matplotlib.colors import LogNorm
plt.figure(figsize=(10,10))
plt.imshow(image, cmap='gray',norm=LogNorm())
for con in catalogue['contour']:
    plt.plot(con[:, 1], con[:, 0])
plt.xlim(200,800)
plt.ylim(200,800)
plt.savefig('images/sample_gal.png')
#plt.show()

![image](images/sample_gal.png)